In [1]:
import numpy as np

np.random.seed(204)

# Problem 2

In [2]:
def arcc_price(c, n):
    """ Calculate the value of an autocallable reverse convertible contract
    using the Monte-Carlo method

    Parameters
    ----------
    c : float
        The annual coupon
    n : int
        The number of simulations

    Returns
    -------
    ev : float
        The simulated value of the contract

    """

    # Initialize the parameters of the contract
    s0, r, sigma, k, t, m, i0 = 100, 0.04, 0.3, 55, 1, 4, 2
    # Calculate the coupon at each payment
    ci = c * t / m
    # Calculate the time interval
    h = t / m
    # Simulate the stock prices
    s_normal = np.random.normal(size=(m, n))
    stock_price = np.exp((r - sigma ** 2 / 2) * h +
                         sigma * np.sqrt(h) * s_normal)
    stock_price = np.cumprod(stock_price, axis=0) * s0
    # Calculate tau
    tau = stock_price[i0 - 1:, :] >= s0
    tau = np.where(tau.any(axis=0), tau.argmax(axis=0) + i0, m)
    # Calculate the no-arbitrage price of the contract
    # Calculate the coupon for different tau's
    coupon = np.array([np.exp(-r * i * h) * ci for i in range(1, m + 1)])
    coupon = np.cumsum(coupon)
    # Calculate the price for each simulation
    v = coupon[tau - 1] + np.where(
        tau < m, np.exp(-r * tau * h) * s0, np.exp(-r * t) * np.where(
            stock_price[-1, :] > k, s0, stock_price[-1, :]))
    # Calculate the mean
    ev = np.mean(v)

    return ev


In [3]:
# Print the result
print('The contract price when c = 5.64%: {:.2f}'.format(
    arcc_price(5.64, int(1e8))))

The contract price when c = 5.64%: 100.00


# Problem 3

In [4]:
def reinforce_function(func, n, *args):
    """ Repeat a function with certain parameters several times and return the
    mean result

    Parameters
    ----------
    func : function
        The function to be repeated
    n : int
        The repeating time
    args : tuple
        The parameters for the function

    Returns
    -------
    mean_value : float
        The mean value of n times running
    """

    value = []
    for i in range(n):
        value.append(func(*args))
    mean_value = np.mean(value)

    return mean_value


In [5]:
def loc_vol_option(step, n):
    """Calculate the option price when the stock price follows a local
    volatility model

    Parameters
    ----------
    step : int
        The number of steps to calculate the SDE in each simulation
    n : int
        The number of simulations

    Returns
    -------
    ec : float
        The fair value of the call option at present
    """

    # Initialize the parameters of the contract and the stock
    s0, k, r, t = 100, 110, 0.05, 1
    delta = t / step

    # Define the function for calculating volatility
    def sigma(st, t):
        return np.power((100 / st), 0.3) / (2 * np.exp(t))

    # Generate standard normal random variables
    z = np.random.normal(size=(n, step))
    # Initialize a container to store the price
    s = np.zeros(shape=(n, step + 1))
    s[:, 0] = s0
    # Use the Euler Scheme to simulate the SDE
    for i in range(step):
        s[:, i + 1] = (1 + r * delta) * s[:, i] + sigma(s[:, i], i * delta) * \
                      s[:, i] * np.sqrt(delta) * z[:, i]
    # Calculate the mean of expected values
    c = np.maximum(s[:, step] - k, 0)
    ec = np.mean(c) * np.exp(-r * t)

    return ec


In [6]:
# Print the result
print('Option Price: {:.1f}'.format(
    reinforce_function(loc_vol_option, 10, 252, int(1e6))))

Option Price: 11.0


# Problem 4

In [7]:
def lookback_call_option(m, step, n):
    """

    Parameters
    ----------
    m : int
        The number of time intervals for observation
    step : int
        The number of steps between two observation points in each simulation
    n : int
        The number of simulations

    Returns
    -------
    ec : float
        The fair value of the call option at present
    """

    # Initialize parameters for stock prices
    s0, v0, r, kappa, theta, sigma_v, rho = 100, 0.04, 0.03, 2, 0.04, 0.5, -0.7
    # Initialize parameters for the option
    t, k = 0.5, 120
    # Calculate total steps and time intervals
    n_step = m * step
    delta = t / n_step
    # Generate standard normal random variables
    z1, z2 = np.random.normal(size=(2, n, n_step))
    # Using Cholesky decomposition to generate correlated normal r.v.s
    zv, zs = z1, rho * z1 + np.sqrt(1 - rho ** 2) * z2
    # Initialize containers to store volatility's and prices
    v = np.zeros(shape=(n, n_step + 1))
    s = np.zeros(shape=(n, n_step + 1))
    v[:, 0] = v0
    s[:, 0] = s0
    # Use the Euler Scheme to simulate the SDE
    for i in range(n_step):
        v[:, i + 1] = v[:, i] + kappa * (theta - np.maximum(v[:, i], 0)) * \
            delta + sigma_v * np.sqrt(
            np.maximum(v[:, i], 0) * delta) * zv[:, i]
        s[:, i + 1] = s[:, i] * np.exp((r - v[:, i] / 2) * delta + np.sqrt(
            np.maximum(v[:, i], 0) * delta) * zs[:, i])
    # Calculate observation indices
    observation_index = [i * step for i in range(1, m + 1)]
    # Calculate the option's price
    c = np.max(np.maximum(s[:, observation_index] - k, 0), axis=1)
    ec = np.mean(c) * np.exp(-r * t)

    return ec


In [8]:
# Print the result
print('Option Price when m = 3: {:.1f}'.format(
    lookback_call_option(3, 84, int(1e6))))
print('Option Price when m = 6: {:.1f}'.format(
    lookback_call_option(6, 42, int(1e6))))
print('Option Price when m = 12: {:.1f}'.format(
    lookback_call_option(12, 21, int(1e6))))

Option Price when m = 3: 0.3
Option Price when m = 6: 0.3
Option Price when m = 12: 0.3


# Problem 5

In [9]:
def reset_strike_option(step, n):
    """Calculate the reset strike option of call type price when the stock
    price follows a variance gamma process

    Parameters
    ----------
    step : int
        The number of steps to calculate the SDE in each simulation
    n : int
        The number of simulations

    Returns
    -------
    ec : float
        The fair value of the reset strike option at present
    """
    # Initialize parameters for stock prices
    theta, sigma, nu, s0, r, delta = -0.1, 0.2, 0.2, 100, 0.03, 0
    # Initialize parameters for the option
    k, t1, t2 = 100, 0.25, 0.5
    # Calculate the time interval and omega
    h = t2 / step
    omega = np.log(1 - theta * nu - sigma ** 2 * nu / 2) / nu
    # Generate standard normal random variables and gamma random variables
    z = np.random.normal(size=(n, step))
    g = np.random.gamma(shape=h / nu, scale=nu, size=(n, step))
    x = theta * g + sigma * np.sqrt(g) * z
    # Initialize a container to store stock prices
    s = np.zeros(shape=(n, step + 1))
    s[:, 0] = s0
    # Simulate the stock price
    for i in range(step):
        s[:, i + 1] = s[:, i] * np.exp((r - delta + omega) * h + x[:, i])
    # Calculate the reset date index
    reset_index = int(np.round(step * t1 / t2))
    # Calculate the mean of expected payoffs
    c = np.maximum(s[:, step] - np.minimum(k, s[:, reset_index]), 0)
    ec = np.mean(c) * np.exp(-r * t2)

    return ec


In [10]:
# Print the result
print('Option Price: {:.2f}'.format(
    reinforce_function(reset_strike_option, 100, 1638, int(1e5))))

Option Price: 7.11
